# Creating Polarity Dictionary

극성사전을 만든다. 

우선 한국은행 의사록, 네이버 금융 증권사 채권보고서, 뉴스 데이터에서 각 일자, 문서별로 ngram 리스트를 뽑아낸 데이터를 불러온다. 

추가적으로 일일 콜금리 데이터를 불러온다. 이 때, 콜금리 데이터는 30일 전과 비교하여 당일 콜금리가 증가했는지 감소했는지 레이블 되어있다. 이를 토대로 금리가 올랐다면 hawkish, 내렸다면 dovish로 가정한다. 

의사록, 채권보고서, 뉴스 데이터 모두 date, ngrams 컬럼만 뽑아낸 뒤 일자별로 콜금리 증감 여부를 merge한다. 

이제 어떤 ngrams (list)가 hawkish하고 dovish한지 레이블 되었기 때문에 일자 상관없이 각 ngram 하나하나의 hawkish 빈도와 dovish 빈도, 그리고 total (= hawkish + dovish) 빈도를 표시한다. 

이제 Naive Bayes Classifier의 가정 하에 각 ngram의 베이즈확률을 구하고 Polarity Score을 구할 수 있다. 

In [34]:
import pandas as pd
import numpy as np

import multiprocessing as mp

## Open data and merge them 

In [2]:
call_rates_df = pd.read_pickle('./data_files/BOK_call_rates.pkl')
call_rates_df.head()

,date,call_rate,call_rate_30daysago,change
31,2004-02-01,3.78,3.77,1
32,2004-02-02,3.78,3.77,1
33,2004-02-03,3.77,3.77,0
34,2004-02-04,3.78,3.76,1
35,2004-02-05,3.79,3.74,1


In [3]:
minutes_df = pd.read_pickle('./data_files/BOK_minutes/pkl/minutes_ngram.pkl')
minutes_df.date = pd.to_datetime(minutes_df.date, format='%Y-%m-%d')
minutes_df = minutes_df
minutes_df.head()

,date,minutes,Foreign Currency,Financial Markets,ngrams
0,2005-06-09,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...,일부 위원은 M1이 실물경제와 어느 정도의 시차를 두고 밀접한 연관성을 지니는지...,"[경제/NNG, 경제/NNG, 비해/VV, 특별/NNG, 요인/NNG, 불구/NNG..."
1,2005-06-23,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,,,[]
2,2005-07-07,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 6월 들어 국제유가가 크게 오르고 원화와 엔화간 동조화 현상이 뚜렷...,일부 위원은 단기적으로 대출이 급격히 증가 한 후에는 반드시 조정이 이어지고 민간...,"[국제/NNG, 유가/NNG, 오르/VV, 원화/NNG, 화간/NNG, 동조/NNG..."
3,2005-08-11,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 위안화 절상 이후 아시아 통화중 우리나라 원화의 절상폭이 가장 컸다...,일부 위원은 최근 장기시장금리 상승에도 불 구하고 기업 자금조달은 오히려 확대되고...,"[위안/NNG, 절상/NNG, 원화/NNG, 절상/NNG, 가장/MAG, 추가/NN..."
4,2005-09-08,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 최근 인도네시아 금융불안 사태가 발생한 가운데 국제 환투기세력이 인...,일부 위원은 시장이 제대로 작동한다면 비우 - 5 - 량등급의 회사채 가산금...,"[금융/NNG, 불안사태/NNG, 발생/NNG, 가운데/NNG, 국제/NNG, 환투..."


In [7]:
len(minutes_df)

301

In [4]:
bond_report_df = pd.read_pickle('./data_files/NAVER_bond_reports/pkl/bond_reports_ngram.pkl')
bond_report_df.date = pd.to_datetime(bond_report_df.date, format='%Y-%m-%d')
bond_report_df.head()

,date,title,ngrams
0,2008-04-01,대우증권대우증권 월간 채권투자 (4월).txt,"[채권투자/NNG, 채권시장/NNG, 전망/NNG, 금융시장/NNG, 對應/NNG,..."
1,2008-04-07,대우증권돌아온 박스권.txt,"[돌아온/VV, 박스권/NNG, 하단/NNG, 상단/NNG, 테스트/NNG, 가능성..."
2,2008-04-07,대우증권시간차 공격.txt,"[돌아온/VV, 박스권/NNG, 하단/NNG, 상단/NNG, 테스트/NNG, 가능성..."
3,2008-04-14,대우증권총재의 변신은 무죄다!.txt,"[믹스/NNG, 합의/NNG, 반영/NNG, 예상/NNG, 경제/NNG, 성장률/N..."
4,2008-04-21,대우증권항상 열려는 있는 가능성.txt,"[경기/NNG, 전망/NNG, 분명/MAG, 인플레이션/NNG, 명분/NNG, 결정..."


In [8]:
len(bond_report_df)

3604

In [6]:
ngram_freq_df = minutes_df[['date', 'ngrams']]
ngram_freq_df = ngram_freq_df.append(bond_report_df[['date', 'ngrams']], ignore_index=True)
ngram_freq_df.head()

,date,ngrams
0,2005-06-09,"[경제/NNG, 경제/NNG, 비해/VV, 특별/NNG, 요인/NNG, 불구/NNG..."
1,2005-06-23,[]
2,2005-07-07,"[국제/NNG, 유가/NNG, 오르/VV, 원화/NNG, 화간/NNG, 동조/NNG..."
3,2005-08-11,"[위안/NNG, 절상/NNG, 원화/NNG, 절상/NNG, 가장/MAG, 추가/NN..."
4,2005-09-08,"[금융/NNG, 불안사태/NNG, 발생/NNG, 가운데/NNG, 국제/NNG, 환투..."


In [9]:
len(ngram_freq_df)

3905

In [12]:
ngram_freq_df = ngram_freq_df.merge(call_rates_df[['date', 'change']], how='left', on='date', )
print(len(ngram_freq_df))
ngram_freq_df.head()

3905


,date,ngrams,change
0,2005-06-09,"[경제/NNG, 경제/NNG, 비해/VV, 특별/NNG, 요인/NNG, 불구/NNG...",-1.0
1,2005-06-23,[],-1.0
2,2005-07-07,"[국제/NNG, 유가/NNG, 오르/VV, 원화/NNG, 화간/NNG, 동조/NNG...",-1.0
3,2005-08-11,"[위안/NNG, 절상/NNG, 원화/NNG, 절상/NNG, 가장/MAG, 추가/NN...",1.0
4,2005-09-08,"[금융/NNG, 불안사태/NNG, 발생/NNG, 가운데/NNG, 국제/NNG, 환투...",-1.0


## Get unique ngrams

In [15]:
all_ngrams = []

for li in ngram_freq_df.ngrams:
    all_ngrams += li

In [22]:
unique_ngrams = set(all_ngrams)

## Count unique ngrams' frequency by label (hawkish/dovish)

In [68]:
ngram_freq_dovish_df = ngram_freq_df[ngram_freq_df.change == -1]
ngram_freq_hawkish_df = ngram_freq_df[ngram_freq_df.change == 1]

In [84]:
unique_ngrams_freq_dic = {}

for element in unique_ngrams:
    unique_ngrams_freq_dic[element] = {'hawkish':0, 'dovish':0}

In [85]:
for ngram_list in ngram_freq_dovish_df.ngrams:
    for ngram in ngram_list:
        unique_ngrams_freq_dic[ngram]['dovish'] += 1
    

In [86]:
for ngram_list in ngram_freq_hawkish_df.ngrams:
    for ngram in ngram_list:
        unique_ngrams_freq_dic[ngram]['hawkish'] += 1
    

In [87]:
unique_ngrams_freq_df = pd.DataFrame.from_dict(unique_ngrams_freq_dic, orient='index', columns=['hawkish', 'dovish'])
unique_ngrams_freq_df.head()

,hawkish,dovish
제군/NNG,2,0
이변/NNG,9,7
불확실성/NNG;안전자산/NNG;선호/NNG,2,2
마다/VV,1,0
수요예측/NNG,1769,2290


In [88]:
unique_ngrams_freq_df['total'] = unique_ngrams_freq_df.hawkish + unique_ngrams_freq_df.dovish
unique_ngrams_freq_df.head()

,hawkish,dovish,total
제군/NNG,2,0,2
이변/NNG,9,7,16
불확실성/NNG;안전자산/NNG;선호/NNG,2,2,4
마다/VV,1,0,1
수요예측/NNG,1769,2290,4059
